In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from metrics import *
from data_prepare import testloader
from models import GCNN, AttGNN


device = torch.device("cuda:0") if torch.cuda.is_available() else torch.cuda("cpu")
model = GCNN()
model.load_state_dict(torch.load("../human_features/GCN.pth")) #path to load the model
model.to(device)
model.eval()

(98396, 7)
Size is : 
98396
Length
19679
4920
GCNN Loaded
GCNN(
  (pro1_conv1): GCNConv(1024, 1024)
  (pro1_fc1): Linear(in_features=1024, out_features=128, bias=True)
  (pro2_conv1): GCNConv(1024, 1024)
  (pro2_fc1): Linear(in_features=1024, out_features=128, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (dropout): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
AttGNN Loaded
AttGNN(
  (pro1_conv1): GATConv(1024, 128, heads=1)
  (pro1_fc1): Linear(in_features=128, out_features=128, bias=True)
  (pro2_conv1): GATConv(1024, 128, heads=1)
  (pro2_fc1): Linear(in_features=128, out_features=128, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_featu

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


GCNN(
  (pro1_conv1): GCNConv(1024, 1024)
  (pro1_fc1): Linear(in_features=1024, out_features=128, bias=True)
  (pro2_conv1): GCNConv(1024, 1024)
  (pro2_fc1): Linear(in_features=1024, out_features=128, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (dropout): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
)

In [2]:
import os
import torch
import glob
import numpy as np 
import random
import math
from os import listdir
from os.path import isfile, join
from torch.utils.data import Dataset as Dataset_n
from torch_geometric.data import DataLoader as DataLoader_n
from torch_geometric.data import Data

npy_file = "../human_features/npy_file_new(human_dataset)_test.npy"
processed_dir="../human_features/processed/"
def bump(g):
    return g
    # return Data.from_dict(g.__dict__)

class LabelledDataset(Dataset_n):
    def __init__(self, npy_file, processed_dir):
      self.npy_ar = np.load(npy_file)
      self.processed_dir = processed_dir
      self.protein_1 = self.npy_ar[:,2]
      self.protein_2 = self.npy_ar[:,5]
      self.label = self.npy_ar[:,6].astype(float)
      self.n_samples = self.npy_ar.shape[0]

    def __len__(self):
      return(self.n_samples)

    def __getitem__(self, index):
      prot_1 = os.path.join(self.processed_dir, self.protein_1[index]+".pt")
      prot_2 = os.path.join(self.processed_dir, self.protein_2[index]+".pt")
      # print(prot_1, prot_2)
      # print(glob.glob(prot_1), glob.glob(prot_2))
      #print(f'Second prot is {prot_2}')
      prot_1 = torch.load(glob.glob(prot_1)[0])
      #print(f'Here lies {glob.glob(prot_2)}')
      prot_2 = torch.load(glob.glob(prot_2)[0])
      prot_1 = bump(prot_1)
      prot_2 = bump(prot_2)
      prot_1.x = prot_1.x.to(torch.float32)
      prot_2.x = prot_2.x.to(torch.float32)
      return prot_1, prot_2, torch.tensor(self.label[index])



dataset = LabelledDataset(npy_file = npy_file ,processed_dir= processed_dir)

In [3]:
testloader = DataLoader_n(dataset=dataset, batch_size=4, num_workers=0)

In [4]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from metrics import *
from models import GCNN, AttGNN
from tqdm.auto import tqdm


device = torch.device("cuda:0") if torch.cuda.is_available() else torch.cuda("cpu")
model = GCNN()
model.load_state_dict(torch.load("../human_features/GCN.pth")) #path to load the model
model.to(device)
model.eval()
predictions = torch.Tensor()
labels = torch.Tensor()
with torch.no_grad():
    for prot_1, prot_2, label in tqdm(testloader):
      prot_1 = prot_1.to(device)
      prot_2 = prot_2.to(device)
      #print("H")
      #print(torch.Tensor.size(prot_1.x), torch.Tensor.size(prot_2.x))
      output = model(prot_1, prot_2)
      predictions = torch.cat((predictions, output.cpu()), 0)
      labels = torch.cat((labels, label.view(-1,1).cpu()), 0)
labels = labels.numpy().flatten()
predictions = predictions.numpy().flatten()

GCNN Loaded


  0%|          | 0/1420 [00:00<?, ?it/s]

In [6]:
mse = get_mse(labels, predictions)
# acc = get_accuracy(labels, predictions, 0.5)
# prec = precision(labels, predictions, 0.5)
# sensitivity = sensitivity(labels, predictions,  0.5)
# specificity = specificity(labels, predictions, 0.5)
# f1 = f_score(labels, predictions, 0.5)
# mcc = mcc(labels, predictions,  0.5)
# auroc = auroc(labels, predictions)
# auprc = auprc(labels, predictions)


print(f'mse : {mse}')
# print(f'Accuracy : {acc}')
# print(f'precision: {prec}')
# print(f'Sensititvity :{sensitivity}')
# print(f'specificity : {specificity}')
# print(f'f-score : {f1}')
# print(f'MCC : {mcc}')
# print(f'AUROC: {auroc}')
# print(f'AUPRC: {auprc}')

mse : 0.030343958702492117


In [7]:
np.mean((np.mean(labels) - labels) ** 2)

0.03072402028112409